In [118]:
from skimage.io import imread
from skimage.color import rgb2gray
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy import ndimage


from skimage import measure
from skimage.draw import polygon_perimeter
from skimage.transform import resize

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from sklearn.preprocessing import OneHotEncoder

In [119]:
def read_in_image(image_path, annotation_path):

    df = pd.read_csv(annotation_path, header=None, sep='\t')
    img_x, img_y, img_w, img_h = df.values[0][1], df.values[0][2], df.values[0][3], df.values[0][4]
    label = str(df.values[0][5])

#     print(df)

    img = imread(image_path)

    img_plate = img[img_y:img_y + img_h, img_x:img_x + img_w]

    # print(img_plate)
#     plt.imshow(img)
#     plt.show()
#     plt.imshow(img_plate)
#     plt.show()
    return img_plate, label

In [120]:
def rotate_image(img_plate, binary_inv_otsu):
    draw_hough_line = np.copy(img_plate)

    edges = cv2.Canny(binary_inv_otsu, 50, 200, apertureSize = 3)
#     plt.imshow(edges, cmap=plt.cm.gray)
#     plt.show()

    lines = cv2.HoughLines(edges, 1, np.pi/180, 2)

    for rho,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))

        cv2.line(draw_hough_line,(x1,y1),(x2,y2),(0,0,255),2)
    
    if debug is True:
        plt.imshow(cv2.cvtColor(draw_hough_line, cv2.COLOR_BGR2RGB))
        plt.show()

#     rotated = ndimage.rotate(edges, 180*lines[0][0][1]/3.1415926-90)
#     print(180*lines[0][0][1]/3.1415926)
    if(180*lines[0][0][1]/3.1415926 < 20):
        img_plate_rotated = ndimage.rotate(img_plate, 180*lines[0][0][1]/3.1415926)
    elif(180*lines[0][0][1]/3.1415926 > 160):
        img_plate_rotated = ndimage.rotate(img_plate, 180-180*lines[0][0][1]/3.1415926)
    else:
        img_plate_rotated = ndimage.rotate(img_plate, 180*lines[0][0][1]/3.1415926-90)
#     plt.imshow(rotated, cmap=plt.cm.gray)
#     plt.show()
#     plt.imshow(img_plate_rotated)
#     plt.show()
    return img_plate_rotated

In [121]:
def detection(digits_cropped, model):
    pred_result = ''
    for d in digits_cropped:

        d = np.reshape(d, (1,28,28,1)) 
        out = model.predict(d)

        p = []
        precision = 0
        for i in range(len(out)):
            z = np.zeros(36)
            z[np.argmax(out[i])] = 1.
            precision = max(out[i])
            p.append(z)
        prediction = np.array(p)
    #     print(prediction)

        # Inverse one hot encoding
        letters = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
        classes = []
        for a in letters:
            classes.append([a])
        ohe = OneHotEncoder(handle_unknown='ignore', categorical_features=None)
        ohe.fit(classes)
        pred = ohe.inverse_transform(prediction)

        if precision > 0.1:
#             print('Prediction : ' + str(pred[0][0]) + ' , Precision : ' + str(precision))
            pred_result += str(pred[0][0])
    return pred_result

In [122]:
def find_digits(image):
    if debug is True:
        plt.imshow(image, cmap=plt.cm.gray)
        plt.show()
#     print(image.shape)

    contours = measure.find_contours(image, 0.5)
    # sort contours in the right order: by left_top corner
    sorted_ctrs = sorted(contours, key=lambda contour: np.min(contour[:,1]))

#     fig, ax = plt.subplots()
#     ax.imshow(image, interpolation='nearest', cmap=plt.cm.gray)
#     for n, contour in enumerate(sorted_ctrs):
#         ax.plot(sorted_ctrs[n][:, 1], sorted_ctrs[n][:, 0], linewidth=2)
#     plt.show()

    h = image.shape[0]
    w = image.shape[1]
    area = w * h

    low = 0.01
    up = 0.08
    min_area = area * low
    max_area = area * up
    w_h_ratio = 0.1
    bbox_center_lo = 0.2 * h
    bbox_center_hi = 0.8 * h
    resize_padding = 5
    h_ratio = 0.3

#     print("hi:{}, lo:{}".format(bbox_center_hi, bbox_center_lo))

    with_boxes  = np.copy(img_plate_rotated)

    digits_cropped = []
    last_contour_bbox = [0, 0, 0, 0]
    for n, contour in enumerate(sorted_ctrs):
        Ymin = int(np.min(contour[:,0]))
        Ymax = int(np.max(contour[:,0]))
        Xmin = int(np.min(contour[:,1]))
        Xmax = int(np.max(contour[:,1])) 
    #     print(Xmin)
        w_ = Xmax - Xmin
        h_ = Ymax - Ymin    
        area_ = w_ * h_
        Y_center = (Ymin + Ymax) / 2 

        # we set three.four evaluation criteria to filter out contours that are not digits
        # 1: min_area < area size < max_area
        # 2: width and height ratio > w_h_ratio
        # 3: bbox_center_lo < center point < bbox_center_hi
        # 4: the height of the digit has to be greater than h_ratio * h_image
        if(area_ > min_area and area_ < max_area and 
           w_/h_ > w_h_ratio and h_/w_ > w_h_ratio and 
           Y_center > bbox_center_lo and Y_center < bbox_center_hi and
           h_ > h_ratio*h):
            if(Xmin > last_contour_bbox[0] and Xmax < last_contour_bbox[1] and
               Ymin > last_contour_bbox[2] and Ymax < last_contour_bbox[3]):
                continue
#             print('w_:{}, h_:{}'.format(w_, h_))
#             print('Y_center:{}'.format(Y_center))
            last_contour_bbox = [Xmin, Xmax, Ymin, Ymax]
    
            r = [Ymin, Ymax, Ymax, Ymin, Ymin]
            c = [Xmax, Xmax, Xmin, Xmin, Xmax]
            rr, cc = polygon_perimeter(r, c, with_boxes.shape)
            with_boxes[rr, cc] = [0,0,255]

            digit = image[Ymin:Ymax, Xmin:Xmax]

#             digit = util.invert(digit)

            # make digit to a squared matrix
            # In case height superior than width
            if h_ > w_:
                diff = h_-w_
                if diff % 2 == 0:
                    x1 = np.zeros(shape=(h_, diff//2))
                    x2 = x1
                else:
                    x1 = np.zeros(shape=(h_, diff//2))
                    x2 = np.zeros(shape=(h_, (diff//2)+1))

                digit = np.concatenate((x1, digit, x2), axis=1)

            # In case height inferior than width
            if h_ < w_:
                diff = w_-h_
                if diff % 2 == 0:
                    x1 = np.zeros(shape=(diff//2, w_))
                    x2 = x1
                else:
                    x1 = np.zeros(shape=(diff//2, w_))
                    x2 = np.zeros(shape=((diff//2)+1, w_))
#                 print(digit.shape, x1.shape, x2.shape)
                digit = np.concatenate((x1, digit, x2), axis=0)

            # put paddings around the digit
            digit = np.pad(digit, ((resize_padding, resize_padding),(resize_padding, resize_padding)), 'constant')

#             print(digit[3])
            digit = cv2.resize(digit, (28,28))
            digit = digit.astype('uint8')
#             print(digit[3])
#             if debug is True:
#                 plt.imshow(digit, cmap=plt.cm.gray)
#                 plt.show()

            digits_cropped.append([digit])
    if debug is True:
        plt.imshow(cv2.cvtColor(with_boxes, cv2.COLOR_BGR2RGB))
        plt.show()
    return digits_cropped

In [127]:
import os
import pandas as pd
import time

debug = False

# data_path = "data/benchmarks/endtoend/test"
data_path = "data/benchmarks/seg_and_ocr/usimages"
# data_path = "data/benchmarks/endtoend/us"
label_path = "data/benchmarks/seg_and_ocr/groundtruth.csv"

labels = pd.read_csv(label_path, header=None, usecols=[0,2], names=['file', 'label'])
# print(labels)

total_plates = 0
correct_predictions = 0
incorrect_predictions = 0

physical_devices = tf.config.experimental.list_physical_devices('GPU') 
for physical_device in physical_devices: 
    tf.config.experimental.set_memory_growth(physical_device, True)

model = load_model('classifier_nor_20.h5')

total_t = 0
count = 0

if 'seg_and_ocr' in data_path:
    # Image set A
    ext = 'png'
elif 'endtoend' in data_path:
    # Image set B
    ext = 'jpg'

for file in os.listdir(data_path):

    file_path = os.path.join(data_path, file)
    
    if file.endswith(ext):
        img_path = file_path
        if ext == 'jpg':
            anno_path = file_path.split('.')[0] + '.txt'
            img_plate, plate_number = read_in_image(img_path, anno_path)
        else:
            plate_number = labels.loc[labels['file']==file, 'label'].values[0]
            img_plate = cv2.imread(img_path, cv2.IMREAD_COLOR)
        
        gray = cv2.cvtColor(img_plate, cv2.COLOR_BGR2GRAY) 
        
        image_to_thresh = gray
        # adaptive thresholding
        binary_inv_adaptive = cv2.adaptiveThreshold(image_to_thresh, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 39, 1)
        # OTSU's thresholding
        ret, binary_inv_otsu = cv2.threshold(image_to_thresh, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        
        # rotate the plate image by doing houghline transformation
        img_plate_rotated = rotate_image(img_plate, binary_inv_otsu)
        
        gray_rotated = cv2.cvtColor(img_plate_rotated, cv2.COLOR_BGR2GRAY) 
        
        # adaptive thresholding
        image_to_thresh = gray_rotated
        binary_inv_adaptive = cv2.adaptiveThreshold(image_to_thresh, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 39, 1)
        # OTSU's thresholding
        ret, binary_inv_otsu = cv2.threshold(image_to_thresh, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        
        # in case the binary_inv_otsu image is in white background, we inverse it again
        total_pixels = binary_inv_otsu.shape[0] * binary_inv_otsu.shape[1]
        total_white = cv2.countNonZero(binary_inv_otsu)
        total_black = total_pixels - total_white
        if(total_white > total_black):
            binary_inv_otsu = cv2.bitwise_not(binary_inv_otsu)
        
        digits_cropped = find_digits(binary_inv_otsu)

#         plt.imshow(img_plate, interpolation='nearest', cmap=plt.cm.gray)
#         plt.show()
        t0 = time.process_time()
        prediction = detection(digits_cropped, model)
        t1 = time.process_time()
        total_t += t1 - t0
        print('file: ' + file)
        print('Prediction: ' + prediction)
        print('Truth: ' + plate_number)
        if prediction == plate_number:
            correct_predictions += 1
        else:
            incorrect_predictions += 1
        total_plates += 1
        print('Detection accuracy: {}/{} = {}'.format(correct_predictions, total_plates, correct_predictions/total_plates))
        print('===================================================')
        count += 1

print('Total time: {} sec.'.format(total_t))
print('Average time: {} sec.'.format((total_t)/count))

file: nj754.png
Prediction: UAAA7E
Truth: UAA47C
Detection accuracy: 0/1 = 0.0
file: ia1387.png
Prediction: 444W00
Truth: 444NOQ
Detection accuracy: 0/2 = 0.0
file: or1509.png
Prediction: UZP660
Truth: UZP660
Detection accuracy: 1/3 = 0.3333333333333333
file: me542.png
Prediction: U4303HI
Truth: 4303HI
Detection accuracy: 1/4 = 0.25
file: in1184.png
Prediction: IAMYTEXT
Truth: ANYTEXT
Detection accuracy: 1/5 = 0.2
file: nj104.png
Prediction: KLX41E
Truth: KLX41C
Detection accuracy: 1/6 = 0.16666666666666666
file: id1280.png
Prediction: WMEKQ
Truth: IDAH0HI
Detection accuracy: 1/7 = 0.14285714285714285
file: ar810.png
Prediction: N0X226
Truth: NOX226
Detection accuracy: 1/8 = 0.125
file: va1523.png
Prediction: IKGV5561
Truth: KGV5561
Detection accuracy: 1/9 = 0.1111111111111111
file: md1304.png
Prediction: 0026
Truth: 0026
Detection accuracy: 2/10 = 0.2
file: dc1415.png
Prediction: CX4887
Truth: CX4887
Detection accuracy: 3/11 = 0.2727272727272727
file: ut427.png
Prediction: 8872B
Truth

file: fl234.png
Prediction: H507VU
Truth: H507VU
Detection accuracy: 18/58 = 0.3103448275862069
file: nc812.png
Prediction: 3887
Truth: 3887
Detection accuracy: 19/59 = 0.3220338983050847
file: ak399.png
Prediction: FGJ35
Truth: FGJ235
Detection accuracy: 19/60 = 0.31666666666666665
file: ri649.png
Prediction: 713731I
Truth: 713731
Detection accuracy: 19/61 = 0.3114754098360656
file: tx1552.png
Prediction: WBRRK3W
Truth: DB8R369
Detection accuracy: 19/62 = 0.3064516129032258
file: va3.png
Prediction: LLA1000
Truth: LLA1000
Detection accuracy: 20/63 = 0.31746031746031744
file: vt370.png
Prediction: 8RRRR
Truth: BRRRR
Detection accuracy: 20/64 = 0.3125
file: va1299.png
Prediction: QCN1309
Truth: CN1309
Detection accuracy: 20/65 = 0.3076923076923077
file: ny705.png
Prediction: ABC2345
Truth: ABC2345
Detection accuracy: 21/66 = 0.3181818181818182
file: ia537.png
Prediction: 576PB0
Truth: 576PBQ
Detection accuracy: 21/67 = 0.31343283582089554
file: il804.png
Prediction: 
Truth: 5415791
Dete

file: id688.png
Prediction: 4
Truth: 71227
Detection accuracy: 34/114 = 0.2982456140350877
file: id1191.png
Prediction: 
Truth: 8862
Detection accuracy: 34/115 = 0.2956521739130435
file: wy1235.png
Prediction: 248
Truth: 248
Detection accuracy: 35/116 = 0.3017241379310345
file: az1193.png
Prediction: A
Truth: 4YA411
Detection accuracy: 35/117 = 0.29914529914529914
file: ok556.png
Prediction: N093G0K1
Truth: 093GOK
Detection accuracy: 35/118 = 0.2966101694915254
file: ia760.png
Prediction: 78B1BZ
Truth: 7881BZ
Detection accuracy: 35/119 = 0.29411764705882354
file: pa411.png
Prediction: GTW8445
Truth: GTM8445
Detection accuracy: 35/120 = 0.2916666666666667
file: wi291.png
Prediction: 6I6FAF
Truth: 616FAF
Detection accuracy: 35/121 = 0.2892561983471074
file: fl227.png
Prediction: FL00
Truth: FLNGP00
Detection accuracy: 35/122 = 0.28688524590163933
file: id991.png
Prediction: 1SZ
Truth: 108565
Detection accuracy: 35/123 = 0.2845528455284553
file: dc188.png
Prediction: 000002
Truth: 800002


file: nj1428.png
Prediction: PLT65K
Truth: PLT65K
Detection accuracy: 55/170 = 0.3235294117647059
file: pa845.png
Prediction: DVK0943
Truth: DVK0943
Detection accuracy: 56/171 = 0.32748538011695905
file: ia396.png
Prediction: J051H
Truth: J0518
Detection accuracy: 56/172 = 0.32558139534883723
file: il608.png
Prediction: FI
Truth: FAMILY
Detection accuracy: 56/173 = 0.3236994219653179
file: vt108.png
Prediction: BKM119
Truth: BKW119
Detection accuracy: 56/174 = 0.3218390804597701
file: al1247.png
Prediction: 47A434F
Truth: 47A434F
Detection accuracy: 57/175 = 0.32571428571428573
file: ok48.png
Prediction: FBHX476
Truth: FBH476
Detection accuracy: 57/176 = 0.32386363636363635
file: vt706.png
Prediction: 00000
Truth: 0
Detection accuracy: 57/177 = 0.3220338983050847
file: or1623.png
Prediction: SK117
Truth: SKT517
Detection accuracy: 57/178 = 0.3202247191011236
file: or822.png
Prediction: Y03I
Truth: YDR431
Detection accuracy: 57/179 = 0.31843575418994413
file: va1463.png
Prediction: ZUC7

file: mn1292.png
Prediction: 903GZ
Truth: 903GZR
Detection accuracy: 71/225 = 0.31555555555555553
file: co122.png
Prediction: I000XXX
Truth: 000XXX
Detection accuracy: 71/226 = 0.3141592920353982
file: nd99.png
Prediction: HCA520F
Truth: HCA520
Detection accuracy: 71/227 = 0.31277533039647576
file: nc541.png
Prediction: CUR4HE0
Truth: CURE4HD
Detection accuracy: 71/228 = 0.31140350877192985
file: az589.png
Prediction: M1627
Truth: W16627
Detection accuracy: 71/229 = 0.31004366812227074
file: id860.png
Prediction: 1AILI3
Truth: 1ADL132
Detection accuracy: 71/230 = 0.30869565217391304
file: id825.png
Prediction: 
Truth: MNW235
Detection accuracy: 71/231 = 0.30735930735930733
file: nv679.png
Prediction: 599HAR
Truth: 599MAR
Detection accuracy: 71/232 = 0.30603448275862066
file: nh936.png
Prediction: 26358I9
Truth: 2635819
Detection accuracy: 71/233 = 0.30472103004291845
file: nd1372.png
Prediction: 1
Truth: GXJ915
Detection accuracy: 71/234 = 0.3034188034188034
file: la237.png
Prediction:

...

file: fl1167.png
Prediction: 7164HF
Truth: 7164HF
Detection accuracy: 180/501 = 0.3592814371257485
file: la787.png
Prediction: MRM33I
Truth: NRM331
Detection accuracy: 180/502 = 0.35856573705179284
file: ca740.png
Prediction: 5ALN015
Truth: 5ALN015
Detection accuracy: 181/503 = 0.35984095427435386
file: nh1411.png
Prediction: 2A9A26A
Truth: 2494264
Detection accuracy: 181/504 = 0.35912698412698413
file: wy369.png
Prediction: 4J
Truth: 874J
Detection accuracy: 181/505 = 0.3584158415841584
file: co1018.png
Prediction: BU4137
Truth: ABU4137
Detection accuracy: 181/506 = 0.3577075098814229
file: ny466.png
Prediction: FAA4563
Truth: FAA4563
Detection accuracy: 182/507 = 0.358974358974359
file: dc1008.png
Prediction: DP7160
Truth: DP7160
Detection accuracy: 183/508 = 0.36023622047244097
file: hi1134.png
Prediction: MZA275
Truth: MZA275
Detection accuracy: 184/509 = 0.3614931237721022
file: tx299.png
Prediction: CK3H547I
Truth: CK3H547
Detection accuracy: 184/510 = 0.3607843137254902
file: ct

file: nc1579.png
Prediction: PYFQ595
Truth: KYP4595
Detection accuracy: 207/556 = 0.3723021582733813
file: wy1514.png
Prediction: 1321
Truth: 1321
Detection accuracy: 208/557 = 0.3734290843806104
file: hi157.png
Prediction: HXM691
Truth: HXM691
Detection accuracy: 209/558 = 0.37455197132616486
file: nv1624.png
Prediction: 212UJW
Truth: 212UJM
Detection accuracy: 209/559 = 0.3738819320214669
file: tx39.png
Prediction: EBB9
Truth: BB9H906
Detection accuracy: 209/560 = 0.3732142857142857
file: pa1652.png
Prediction: LDYL4905
Truth: DYL4905
Detection accuracy: 209/561 = 0.37254901960784315
file: ca1592.png
Prediction: I5PPP06L
Truth: 5PPP064
Detection accuracy: 209/562 = 0.3718861209964413
file: va445.png
Prediction: 279AX
Truth: 1279AX
Detection accuracy: 209/563 = 0.37122557726465366
file: nh1557.png
Prediction: FIXI69C
Truth: FIX169C
Detection accuracy: 209/564 = 0.37056737588652483
file: nh228.png
Prediction: 160952
Truth: 760952
Detection accuracy: 209/565 = 0.36991150442477877
file: 

file: ri165.png
Prediction: AB634
Truth: GB634
Detection accuracy: 226/611 = 0.3698854337152209
file: nc1202.png
Prediction: LXK5176
Truth: LXK5176
Detection accuracy: 227/612 = 0.3709150326797386
file: id913.png
Prediction: UPDCRK
Truth: UPDCRIK
Detection accuracy: 227/613 = 0.3703099510603589
file: hi597.png
Prediction: QAN91L
Truth: AA974
Detection accuracy: 227/614 = 0.3697068403908795
file: ma805.png
Prediction: MRLDTC
Truth: WRLDTC
Detection accuracy: 227/615 = 0.36910569105691055
file: la701.png
Prediction: LFMQ397
Truth: LFM397
Detection accuracy: 227/616 = 0.3685064935064935
file: hi841.png
Prediction: KZF827
Truth: KZF827
Detection accuracy: 228/617 = 0.3695299837925446
file: nd916.png
Prediction: OBB63
Truth: BB63
Detection accuracy: 228/618 = 0.36893203883495146
file: vt1092.png
Prediction: UUISY
Truth: GUTSY
Detection accuracy: 228/619 = 0.3683360258481422
file: il616.png
Prediction: H103801
Truth: H103801
Detection accuracy: 229/620 = 0.36935483870967745
file: ny986.png
P

file: va1365.png
Prediction: WSAMPLE
Truth: SAMPLE
Detection accuracy: 247/666 = 0.3708708708708709
file: va1037.png
Prediction: P0P01Z1
Truth: P0PQIZ
Detection accuracy: 247/667 = 0.37031484257871067
file: la163.png
Prediction: 0IT643
Truth: OIT643
Detection accuracy: 247/668 = 0.36976047904191617
file: tx128.png
Prediction: KK
Truth: BH8G151
Detection accuracy: 247/669 = 0.3692077727952167
file: ar883.png
Prediction: 087LAN
Truth: 087LAN
Detection accuracy: 248/670 = 0.3701492537313433
file: la275.png
Prediction: TAF389
Truth: TAF389
Detection accuracy: 249/671 = 0.3710879284649776
file: id1405.png
Prediction: FELEEH
Truth: 8BL8976
Detection accuracy: 249/672 = 0.3705357142857143
file: ak721.png
Prediction: FHG521
Truth: FHG521
Detection accuracy: 250/673 = 0.37147102526002973
file: ky1217.png
Prediction: 88508
Truth: BBG50B
Detection accuracy: 250/674 = 0.37091988130563797
file: il330.png
Prediction: 5L
Truth: K545614
Detection accuracy: 250/675 = 0.37037037037037035
file: mn723.png

file: la1356.png
Prediction: UZTHE4CI
Truth: UZTHE4C
Detection accuracy: 267/721 = 0.37031900138696255
file: nj600.png
Prediction: ZUP3EA
Truth: ZUP36A
Detection accuracy: 267/722 = 0.3698060941828255
file: id260.png
Prediction: WW1UE
Truth: 1A2E468
Detection accuracy: 267/723 = 0.36929460580912865
file: ia1025.png
Prediction: 469RWW
Truth: 469BWW
Detection accuracy: 267/724 = 0.3687845303867403
file: ny524.png
Prediction: FAE6137
Truth: FAE6137
Detection accuracy: 268/725 = 0.3696551724137931
file: de1608.png
Prediction: KHP7562
Truth: HP7562
Detection accuracy: 268/726 = 0.3691460055096419
file: nj552.png
Prediction: YAH87F
Truth: YAH87F
Detection accuracy: 269/727 = 0.3700137551581843
file: pa534.png
Prediction: ASUKA
Truth: ASUKA
Detection accuracy: 270/728 = 0.3708791208791209
file: ut993.png
Prediction: Z69
Truth: Z697BP
Detection accuracy: 270/729 = 0.37037037037037035
file: ga31.png
Prediction: APN5740
Truth: APM5740
Detection accuracy: 270/730 = 0.3698630136986301
file: ne280.